In [3]:
#eliminacion de variables irrelevantes

def descarta(m, c, e):
    lista = c+e
    descartados = []
    padres = []
    for l in lista:
        subgrafo = Modelo_residentevil.subgraph(networkx.ancestors(Modelo_residentevil, l))
        for s in subgrafo:
            padres.append(s)
                          
    for n in m.nodes():
        if n not in lista and n not in padres:
            descartados.append(n)
                  
    return descartados

In [4]:
#codigo para generar una muestra aleatoria
#--------------------------------------------------------------------------
import random

def seleccionar_probabilidad(cpd, valor, evidencia):
    padres = cpd.evidence if cpd.evidence else []
    valores_evidencia = tuple(evidencia[var] for var in padres)
    return cpd.values[valor][valores_evidencia] #me devuelve los valores de la distribucion de probabilidad conjunta 
            #(los numeros de la tabla de arriba) los valores de la evidencia vienen dado por los valores de los padres
    
def generar_valor_aleatorio(cardinalidad, probabilidades):  #para hacer lo de la ruleta. Poner en una linea de  0 a 1 
    p = random.random()                                     #todo lo que tengo respetando la cardinalidad 
    acumuladas = 0                                          #(la probabilidad de que salga)
    for valor in range(cardinalidad):
        acumuladas += probabilidades[valor] 
        if p <= acumuladas:
            return valor

In [5]:
def muestra_aleatoria(red):
    variables = networkx.topological_sort(red)
    muestra = {}
    for variable in variables:
        tabla = Modelo_alarma.get_cpds(variable)
        cardinalidad = tabla.variable_card
        probabilidades = [seleccionar_probabilidad(tabla,i,muestra)
                         for i in range(cardinalidad)]
        valor = generar_valor_aleatorio(cardinalidad,probabilidades)
        muestra[variable] = valor
    return muestra
#---------------------------------------------------------------------------------------

In [ ]:
def verosimilitud(modelo, consulta, evidencias):
    descartes = descarta(modelo, consulta, evidencias)
    nodos = modelo.nodes()
    candidatas = nodos - descartes - consulta - evidencias
    top = 0
    node = None 
    for n in candidatas:
        aleatorio = nodos - consulta - evidencias - n
        candPos = n #nodo positivo
        candNeg = n #nodo en negativo
        valorPos = valorNeg = 0
        contador = 1000
        tupla = BayesianModel()
            for g in nodos:
                if g is not n:
                    tupla.add_node('g')
       
        while (contador > 0):
            
            tuplaPos = muestra_aleatoria(tupla)
            tuplaNeg = muestra_aleatoria(tupla)
            tuplaPos.add_node('n') #positivo
            tuplaNeg.add_node('n') #negativo. no se como hacerlo
            
            #calcular valor de ambas tuplas
            calculoPos = calcularValor(tuplaPos) #codigo a parte. No se como. Nodos tienen valor como atributo? 
            caltuloNeg = calcuarValor(tuplaNeg) #calcular un valor con codigo llano pero solo afectarian las evidencias  
            valorPos = valorPos + calculoPos
            valorNeg = valorNeg + calculoNeg
            
            contador = contador -1
        vPos = valorPos/1000
        vNeg = valorneg/1000
        k = abs(vPos-vNeg)
        if(k> top):
            top = k
            node = n
    
    return node
            